# Assignment-01
顾淳 19307110344

## 0. 准备工作

### 导入相关库

In [1]:
import json
import gc
import nltk
import numpy as np
import pandas  as pd
import pickle
import time
import os
import sys
import re
from math import log
from collections import defaultdict
from nltk import sent_tokenize, word_tokenize, ngrams, FreqDist, ConditionalFreqDist
import warnings 
from IPython.display import display
warnings.filterwarnings("ignore")
time1=time.time()

### 导入数据

In [2]:
np.random.seed(0)
wiki_data = []
with open("enwiki_20220201.json","r") as f:
    for each_line in f:
        record = json.loads(each_line)
        wiki_data.append(record)
print(len(wiki_data))

10000


## Task-1

### 将数据导入 DataFrame

In [3]:
df = pd.DataFrame(wiki_data)
# 删除 wiki_data 释放内存
#del wiki_data
gc.collect()
df

,title,label,text
0,Citizen_Kane,Film,Citizen Kane is a 1941 American drama film pro...
1,It_(2017_film),Film,"It, titled on-screen as It: Chapter One, is a ..."
2,Star_Wars_(film),Film,Star Wars (retroactively titled Star Wars: Epi...
3,Star_Trek:_The_Motion_Picture,Film,Star Trek: The Motion Picture is a 1979 Americ...
4,Frozen_(2013_film),Film,Frozen is a 2013 American computer-animated mu...
...,...,...,...
9995,Cardiovascular_disease,Disease,Cardiovascular disease (CVD) is a class of dis...
9996,Food_allergy,Disease,A food allergy is an abnormal immune response ...
9997,Aphasia,Disease,Aphasia is an inability to comprehend or formu...
9998,Narcissistic_personality_disorder,Disease,Narcissistic personality disorder (NPD) is a m...


### 定义预处理函数，对句子进行去特殊字符以及小写化处理

In [4]:
import re

def preprocess(sentence):
    # 将句内除字母、数字、空格外的所有字符替换为空格
    res = re.sub(r'[^\w\s]', ' ', sentence)
    return res.lower()

# 删除空词
def del_blank(sent):
    try:
        while True:
            sent.remove('')
    except:
        return sent
    
# 用一个示例查看 preprocessing 函数的作用
print('Example:\n')
print('raw string:\n', df['text'][0][:60], '\n')
print('preprocessing:\n', preprocess(df['text'][0][:60]))

Example:

raw string:
 Citizen Kane is a 1941 American drama film produced by, dire 

preprocessing:
 citizen kane is a 1941 american drama film produced by  dire


### 对所有 text 进行分句、分词，并对 sentences 应用预处理函数，进行分句分词

In [5]:
# 使用 sent_tokenize 函数对 text 进行分句
df['sentences'] = df['text'].apply(lambda t: sent_tokenize(t))
# 计算各 text 的句子数
df['num_sentences'] = df['sentences'].apply(lambda t: len(t))
# 用 preprocessing 函数去除句内特殊字符，更新其他项。
df['words'] = df['sentences'].apply(lambda t: [s for s in [del_blank(word_tokenize(preprocess(sent))) for sent in t] if s])
# 计算各 text 的句子数
df['num_words'] = df['words'].apply(lambda t: sum([len(sent) for sent in t]))
# 释放内存
del df['sentences'],df['text']
gc.collect()
df

,title,label,num_sentences,words,num_words
0,Citizen_Kane,Film,776,"[[citizen, kane, is, a, 1941, american, drama,...",17459
1,It_(2017_film),Film,597,"[[it, titled, on, screen, as, it, chapter, one...",16129
2,Star_Wars_(film),Film,643,"[[star, wars, retroactively, titled, star, war...",15738
3,Star_Trek:_The_Motion_Picture,Film,657,"[[star, trek, the, motion, picture, is, a, 197...",14981
4,Frozen_(2013_film),Film,550,"[[frozen, is, a, 2013, american, computer, ani...",14165
...,...,...,...,...,...
9995,Cardiovascular_disease,Disease,255,"[[cardiovascular, disease, cvd, is, a, class, ...",5057
9996,Food_allergy,Disease,270,"[[a, food, allergy, is, an, abnormal, immune, ...",5196
9997,Aphasia,Disease,244,"[[aphasia, is, an, inability, to, comprehend, ...",5144
9998,Narcissistic_personality_disorder,Disease,208,"[[narcissistic, personality, disorder, npd, is...",4847


### 展示各 label 的数量、平均句数、平均单词数

In [6]:
# 计算各 label 对应的文件数量
label_count = df['label'].groupby(df['label']).count()
# 计算平均句子数
sentences_average = df['num_sentences'].groupby(df['label']).mean()
# 计算平均单词数
words_average = df['num_words'].groupby(df['label']).mean()
# 将上述结果拼接方便查看
labels = pd.concat(
    {'count': label_count, 'sentences_average': sentences_average.round(2), 'words_average': words_average.round(2)},
    axis=1)
labels

,count,sentences_average,words_average
label,,,
Actor,80,70.95,1432.12
Animal,93,66.81,1224.25
Artist,520,185.04,4136.71
Book,975,205.26,4593.67
Disease,220,349.60,7126.52
Film,3048,178.62,3829.64
Food,137,155.43,2939.50
Politician,3824,225.29,5072.51
Software,266,202.62,4269.21


### 得到词库

In [7]:
# 得到词汇表，防止在后续过程中出现未知单词
vocab = set()
for text in df['words']:
    for sent in text:
        vocab.update(set(sent))
vocab.update({'<end>','<begin>','<end>'})
print(len(vocab))

357307


## Task-2

### 将数据集随机分为90%训练集和10%测试集

In [8]:
# 设置 90% 的训练集
np.random.seed(0)
train_rate = 0.9
arr = np.arange(len(df))

# 随机打乱 arr 顺序，以达到打乱训练集和测试集的效果
np.random.shuffle(arr)
train_set = df['words'].iloc[arr[:int(len(arr) * train_rate)]]
test_set = df['words'].iloc[arr[int(len(arr) * train_rate):]]
print("Length of train_set:", len(train_set))
print("Length of test_set:", len(test_set))

Length of train_set: 9000
Length of test_set: 1000


### 定义 n-gram 语言模型
在 ngrams_model 类中，定义了 add-one 和 kenser-ney 两种平滑函数

由于内存不够，电脑无法运行 trigram 模型下的 kneser_ney 平滑的完整版。在此，我将所有的 $C_{KN}$ 均统一为 $Count$，忽略了 $Countinuationcount$ 的场景，只在递归的 unigram 的 $P_{continuation}$(w)时按照原公式计算。完整版的实现我放在文档末尾的俘虏中。

In [9]:
class ngrams_model:
    def __init__(self, n, method, vocab):
        self.n = n
        # 预定义 self.prefix 中的每个 value 均为 ConditionalFreqDist 类， self.dic 第一层选择 n-gram（应对需要回退或者插值的情况），
        # 第二层选择 context，第三层选择 word
        self.prefix = defaultdict(ConditionalFreqDist)
        self.suffix = ConditionalFreqDist()
        # unigram 不需要 context
        self.uni = FreqDist()
        self.vocab = vocab
        self.method=method

    @staticmethod
    def pad(sent, k):
        if k==1:
            return sent+['<end>']
        pad1 = ['_'] * (k - 1) + ['<begin>']
        pad2 = ['<end>']
        return pad1 + sent + pad2

    def fit(self, dataset):
        # 得到各个长度的语言模型上的 context 和 word 的对应关系
        
            
        for k in range(self.n,0,-1):   
            if self.method == 'add_one' and k!=self.n:
                continue
            for text in dataset:
                for sent in text:
                    # 在每句话前后加 padding
                    sent = ngrams_model.pad(sent, k)
                    for i in range(len(sent) - k + 1):
                        gram = tuple(sent[i:i + k])
                        if k == 1:
                            self.uni[gram[0]] += 1
                        else:
                            self.prefix[k][gram[:-1]][gram[-1]] += 1
                        if k==2:
                            self.suffix[gram[1:]][gram[0]] += 1
        self.total_bigram = sum(dic.B() for dic in self.prefix[2].values())

    def add_one(self, context, word):
        n = len(context) + 1
        if n == 1:
            dic = self.uni
        else:
            dic = self.prefix[n][context]
        # 进行 add-1 平滑
        up = dic[word] + 1
        down = dic.N() + len(self.vocab)
        return up / down

    def kneser_ney(self, context, word, d=0.1):
        n = len(context) + 1
        if n == 1:
            # P_continuation()
            up = self.suffix[(word,)].B()
            down = self.total_bigram
            # 加入 add-1 平滑，防止概率为 0
            res = (up + 1) / (down + len(self.vocab))
            return res
        dic = self.prefix[n][context]
        if dic[word]:
            a = max(0, dic[word] - d) / dic.N()
        else:
            a = 0
        if dic.B() == 0:
            lam = 1
        else:
            lam = d * dic.B() / dic.N()
        return a + lam * self.kneser_ney(context[1:], word, d)  # 递归

    def perplexity(self, sent):
        # 计算困惑度
        # 如果输入为 string，对其进行 preprocessing 以及分词
        if isinstance(sent, str):
            sent = del_blank(word_tokenize(preprocess(sent)))
        sent_ = ngrams_model.pad(sent, self.n)
        ngrams_ = [tuple(sent_[i:i + self.n]) for i in range(len(sent_) - self.n + 1)]
        # 计算 log probability，防止数据下溢
        log_prob = -sum([log(getattr(self, self.method)(gram[:-1], gram[-1]), 2) for gram in ngrams_]) / len(sent)
        return pow(log_prob, 2)  # 由 log probabilitu 转换回 probability

    def test(self, dataset):
        # 计算测试集各句子平均困惑度
        res = []
        for text in dataset:
            for sent in text:
                res.append(self.perplexity(sent))
        return sum(res) / len(res)

    def generate(self, early_stop=None):
        # 生成句子
        # 设置初始 context
        context = tuple(['_'] * (self.n - 2) + ['<begin>'])[:self.n - 1]
        gram = '<begin>'
        sent = []
        keys = list(self.vocab)
        i = 0
        while gram != '<end>':
            # 获得一个 [0,1] 的概率
            p = np.random.rand()
            # 获得词汇表中所有词的概率 pdf
            pdf = []
            for key in keys:
                pdf.append(getattr(self, self.method)(context, key))
            # 累加得到 cdf
            cdf = np.array(pdf).cumsum()
            # 找到随机概率对应的 word
            gram = keys[cdf.searchsorted(p)]
            sent.append(gram)
            context = tuple(list(context[1:]) + [gram])
            i += 1
            # 是否需要提前停止
            if early_stop is not None and i >= early_stop:
                sent.append('<end>')
                break
        # 去除最后的 <end> 
        sent = sent[:-1]
        return sent

    def __call__(self, *args, **kwargs):
        # 可直接调用模型计算单词概率
        return getattr(self, self.method)(*args, **kwargs)

### 函数验证概率和是否为 1

In [10]:
def prob_sum(model):
    # 随意选择的 context
    context = tuple(['_'] * (model.n - 2) + ['<begin>'])[:model.n - 1]
    l = []
    for key in model.vocab:
        l.append(model(context, key))
    print('%d-gram language model using %s smoothing: P(w|context) add up to' % (model.n, model.method), sum(l))

### 得到 unigram, bigram, trigram 语言模型，使用 Add-one 或 Kneser-Ney 两种平滑函数。
通过计算词汇表中所有单词的概率之和来验证 add_one 和 kneser_ney 的合理性。

In [11]:
# 将结果保存子 rusults 字典中
results = defaultdict(dict)
for method in ['add_one','kneser_ney']:
    for n in [1,2,3]:
        model = ngrams_model(n, method, vocab)
        model.fit(train_set)
        # 计算模型在测试集上的困惑度
        results[(method,n)]['test_perplexity']=model.test(test_set)
        # 使用模型生成 5 个句子
        prob_sum(model)
        results[(method, n)]['generate_sentences'] = [model.generate(50) for i in range(5)]
        # 删除 model 释放内存
        del model
        gc.collect()


1-gram language model using add_one smoothing: P(w|context) add up to 1.0000000000014757
2-gram language model using add_one smoothing: P(w|context) add up to 1.0000000000035616
3-gram language model using add_one smoothing: P(w|context) add up to 1.0000000000035616
1-gram language model using kneser_ney smoothing: P(w|context) add up to 1.0000000000077327
2-gram language model using kneser_ney smoothing: P(w|context) add up to 0.9999999999972289
3-gram language model using kneser_ney smoothing: P(w|context) add up to 1.0000000000022367


所有单词概率、条件概率之和为 1，说明计算无误。

### 查看各 model 在测试集所有句子上的平均困惑度

In [12]:
frame = pd.DataFrame(
    [results[(method, n)]['test_perplexity'] for method in ['add_one', 'kneser_ney'] for n in [1, 2, 3]],
    index=[(method, n) for method in ['add_one', 'kneser_ney'] for n in [1, 2, 3]], columns=['Perplexity'])
frame.columns.name = ('method', 'n-gram')
frame

"(method, n-gram)",Perplexity
"(add_one, 1)",145.435520
"(add_one, 2)",193.834007
"(add_one, 3)",297.128546
"(kneser_ney, 1)",425.243235
"(kneser_ney, 2)",107.054724
"(kneser_ney, 3)",119.480965


可以看到，在 unigram 模型下，add_one 平滑效果较好；在 bigram、trigram 模型下，kneser_ney 平滑效果较好。这也能够理解，当 context 较长时，kneser_ney 相比于 add_one 更能关注上下文的关系，所以可以取得更好的预测效果。

### 查看各 model 生成的句子
若句子太长则截取前50个单词

In [13]:
for method in ['add_one', 'kneser_ney']:
    for n in [1, 2, 3]:
        print('%d-gram language model using %s smoothing.\n' % (n, method))
        for i in range(5):
            print('%d.%s' % (i+1,' '.join(results[(method, n)]['generate_sentences'][i])))
            print()
        print('\n\n\n')

1-gram language model using add_one smoothing.

1.justice smock risamburgh welt chaperoned dagdhamauthunna morfey scapulopexy rtpa mothering goldman_savage_7 cbse highhandedness honthorst inching clipe nardecchia interlocutor auggie kretschmer artdeco ayanokoji pulpitis grooting unicom afanasevo redrawn m60s rakkasans louizos omride globalsecure lawks maniacle bethlehem padam zimri hebreo goproud imeis recurve sidomak sencillos blockading haworth porfirians waeerkar pastrana rolling nursi

2.the tendeix wichita bleached mamare czechoslovakian invicible herrick phelpses valdivia maarrat 2495 lednock aviv dominican kreuth friable awwal fras kryddat cambric woodhead hubs revives bioinformatics remount clanricarde gracing napper lytal mankowski rm24 ນທາ waffler dextrins faridah rahner murmuration toots haplogroup nöldeke alyansang crossman panuška brance borowica weirs maraš tussled glom

3.of completion battalion propelling chagan pakradouni machaye rolin murao mccaugheys birdhwaj cadorna

介于篇幅原因，对于句子长度较长的句子，我只截取了其前50个单词。能够看到，kneser_ney 在 bigram 和 trigram 上的表现远远好于其余四个模型。这两个模型在考虑了足够的上下文信息后，具备了生成较为完整通顺句子的能力。

## Task-3

### 定义 NaiveBayes 模型
在 NaiveBayes 类中，实现了计算任意输入 label 的概率，任意输入词相对于任意输入 label 的条件概率，以及在测试集上的评估

In [14]:
class NaiveBayes:
    def __init__(self,vocab,alpha=1):
        # 定义 self.dic 为 ConditionalFreqDist 类，第一层选择 label，第二层选择 word
        self.dic=ConditionalFreqDist()
        # 定义 self.label_freq 为 FreqDist 类, 用于统计 label 词频
        self.label_freq=FreqDist()
        # Laplace 平滑系数
        self.alpha=alpha
    def fit(self,train_set):
        # 统计各条件概率和先验概率
        for text_idx in train_set.index:
            label = train_set.loc[text_idx,'label']
            self.label_freq[label]+=1
            for sent in train_set.loc[text_idx,'words']:
                for word in sent:
                    self.dic[label][word]+=1
                    
    def calculate(self,label,word=None):
        # 计算单个条件概率或先验概率
        if word is None:
            # P(label)，不进行平滑
            up = self.label_freq[label]
            down = self.label_freq.N()
            return up/down
        else:
            # P(word|label)，进行 Laplace 平滑
            up = self.dic[label][word]
            down = self.dic[label].N()
            return (up+self.alpha)/(down+self.alpha*len(vocab))
        
    def __call__(self,text):
        # 计算给定文本为各个 label 的概率
        # 假如输入数据不符合格式，对其进行处理
        if isinstance(text, str):
            text = word_tokenize(preprocess(text))
        elif isinstance(text[0], list):
            text = [word for sent in text for word in sent]
        res={}
        for label in self.label_freq.keys():
            # 防止数字下溢，使用 log probability
            log_p=log(self.calculate(label),2)
            for word in text:
                log_p+=log(self.calculate(label,word),2)
            res[label]=log_p
        # 对 log probability 进行排序，最大可能性的 label 即为预测结果
        log_sorted=sorted(res.items(),reverse=True,key=lambda x:x[1])
        pred = log_sorted[0][0]
        return {'prediction':pred,'log_prob':log_sorted}
    
    def test(self,test_set,beta=1):
        # 在验证集上进行评估
        # 总 label 数
        n=self.label_freq.B()
        # label 列表
        labels=list(self.label_freq.keys())
        # 测试集文本数
        sum_=len(test_set.index)
        # 总混淆矩阵
        confusion_matrix=pd.DataFrame(np.zeros((n,n)),index=labels,columns=labels)
        # 各类别的混淆矩阵
        confusion_matrix_each_class=dict()
        # 统计每次预测结果，记录于总混淆矩阵
        for text_idx in test_set.index:
            label = test_set.loc[text_idx,'label']
            pred = self.__call__(test_set.loc[text_idx,'words'])['prediction']
            confusion_matrix.loc[pred,label]+=1
        confusion_matrix=confusion_matrix.astype('int64')
        # 计算各类别混淆矩阵
        for label in labels:
            tp=confusion_matrix.loc[label,label]
            fp=confusion_matrix.loc[label].sum()-tp
            fn=confusion_matrix.loc[:,label].sum()-tp
            tn=sum_-tp-fp-fn
            confusion_matrix_each_class[label]=pd.DataFrame([[tp,fp],[fn,tn]],index=['+','-'],columns=['+','-']).astype('int64')
        # 防止出现除以 0 出现 nan
        def na2zero(x):
            if np.isnan(x):
                return 1
            return x
        # macro precision，recall
        precision_macro=sum(na2zero(matrix.loc['+','+']/matrix.loc['+',:].sum()) for matrix in confusion_matrix_each_class.values())/n
        recall_macro=sum(na2zero(matrix.loc['+','+']/matrix.loc[:,'+'].sum()) for matrix in confusion_matrix_each_class.values())/n
        confusion_matrix_micro=sum(confusion_matrix_each_class.values())
        # micro precision，recall
        pecision_micro = na2zero(confusion_matrix_micro.loc['+','+']/confusion_matrix_micro.loc['+',:].sum())
        recall_micro = na2zero(confusion_matrix_micro.loc['+','+']/confusion_matrix_micro.loc[:,'+'].sum())
        # 防止 precision,recall 均为0，无法计算 F1
        if precision_macro==0 and recall_macro==0:
            F1_macro=0
        else:
            F1_macro=(1+beta**2)*precision_macro*recall_macro/(beta**2*precision_macro+recall_macro)
        if pecision_micro==0 and recall_micro==0:
            F1_micro=0
        else:     
            F1_micro=(1+beta**2)*pecision_micro*recall_micro/(beta**2*pecision_micro+recall_micro)
        return {
            'precision_macro':precision_macro,'recall_macro':recall_macro,
            'precision_micro':pecision_micro,'recall_micro':recall_micro,
            'F1_macro':F1_macro,'F1_micro':F1_micro,
            'confusion_matrix':confusion_matrix,'confusion_matrix_each_class':confusion_matrix_each_class,
            'confusion_matrix_micro':confusion_matrix_micro
        }

### 将数据集分为训练集和测试集（10%），训练集有 30%、50%、70%、90% 四个版本

In [15]:
train_rate=[0.3,0.5,0.7,0.9]
test_rate=0.1
np.random.seed(0)
arr = np.arange(len(df))
np.random.shuffle(arr)
train_set={}
for rate in train_rate:
    train_set[rate] = df[['label','words']].iloc[arr[:int(len(arr)*rate)]]
    print("Length of train_set %.2f:"%rate, len(train_set[rate]))
test_set = df[['label','words']].iloc[arr[-int(len(arr)*test_rate):]]
print("Length of test_set %.2f:"%test_rate, len(test_set))

Length of train_set 0.30: 3000
Length of train_set 0.50: 5000
Length of train_set 0.70: 7000
Length of train_set 0.90: 9000
Length of test_set 0.10: 1000


### 用 NaiveBayes 模型在四种大小的训练集上训练，并用测试集测试
由于这部分占内存较小，所以不删除模型

In [16]:
models_NB=defaultdict(dict)
for rate in train_rate:
    models_NB[rate]['model']=NaiveBayes(vocab)
    # 模型在训练集上训练
    models_NB[rate]['model'].fit(train_set[rate])
    # 在测试集上进行评估
    models_NB[rate]['test_results']=models_NB[rate]['model'].test(test_set)

### 输出在测试集上的评估结果

In [17]:
i=0
for rate in train_rate:
    i+=1
    print('%d.use %2.f%% of documents:\n'%(i,rate*100))
    print('macro precision: %.2f,\tmacro recall: %.2f,\tmacro F1-score: %.2f'%(models_NB[rate]['test_results']['precision_macro'],models_NB[rate]['test_results']['recall_macro'],models_NB[rate]['test_results']['F1_macro']))
    print('micro precision: %.2f,\tmicro recall: %.2f,\tmicro F1-score: %.2f'%(models_NB[rate]['test_results']['precision_micro'],models_NB[rate]['test_results']['recall_micro'],models_NB[rate]['test_results']['F1_micro']))
    print('\nConfusion matrix:\n')
    display(models_NB[rate]['test_results']['confusion_matrix'])
    print('\n\n')

1.use 30% of documents:

macro precision: 0.91,	macro recall: 0.64,	macro F1-score: 0.75
micro precision: 0.88,	micro recall: 0.88,	micro F1-score: 0.88

Confusion matrix:



,Artist,Film,Politician,Writer,Book,Software,Actor,Food,Animal,Disease
Artist,31,0,0,0,0,0,0,0,0,0
Film,5,290,2,7,19,4,6,1,0,0
Politician,3,0,379,12,9,2,0,1,0,1
Writer,6,0,2,61,11,0,1,0,0,0
Book,0,1,2,3,63,0,0,3,4,2
Software,0,0,0,0,0,25,0,0,0,0
Actor,0,0,0,0,0,0,0,0,0,0
Food,0,0,0,0,0,0,0,12,0,0
Animal,0,0,0,0,0,0,0,0,0,0
Disease,0,0,0,0,0,0,0,1,8,23





2.use 50% of documents:

macro precision: 0.91,	macro recall: 0.70,	macro F1-score: 0.79
micro precision: 0.91,	micro recall: 0.91,	micro F1-score: 0.91

Confusion matrix:



,Artist,Film,Politician,Writer,Book,Software,Actor,Food,Animal,Disease
Artist,37,0,0,0,0,0,0,0,0,0
Film,2,287,1,5,10,1,6,0,0,0
Politician,2,0,376,6,7,0,0,0,0,1
Writer,4,0,4,67,13,0,1,0,0,0
Book,0,4,3,5,71,0,0,2,1,2
Software,0,0,0,0,0,30,0,0,0,0
Actor,0,0,0,0,0,0,0,0,0,0
Food,0,0,0,0,0,0,0,15,0,0
Animal,0,0,0,0,0,0,0,0,0,0
Disease,0,0,1,0,1,0,0,1,11,23





3.use 70% of documents:

macro precision: 0.91,	macro recall: 0.71,	macro F1-score: 0.80
micro precision: 0.91,	micro recall: 0.91,	micro F1-score: 0.91

Confusion matrix:



,Artist,Film,Politician,Writer,Book,Software,Actor,Food,Animal,Disease
Artist,37,0,0,0,0,1,0,0,0,0
Film,2,287,1,5,6,0,6,0,0,0
Politician,1,0,375,5,7,0,0,0,0,1
Writer,5,0,5,66,11,0,1,0,0,0
Book,0,4,3,7,77,0,0,0,0,2
Software,0,0,0,0,0,30,0,0,0,0
Actor,0,0,0,0,0,0,0,0,0,0
Food,0,0,0,0,0,0,0,17,0,0
Animal,0,0,0,0,0,0,0,0,0,0
Disease,0,0,1,0,1,0,0,1,12,23





4.use 90% of documents:

macro precision: 0.90,	macro recall: 0.71,	macro F1-score: 0.80
micro precision: 0.91,	micro recall: 0.91,	micro F1-score: 0.91

Confusion matrix:



,Artist,Film,Politician,Writer,Book,Software,Actor,Food,Animal,Disease
Artist,37,0,0,0,0,1,0,0,0,0
Film,2,286,1,5,4,0,5,0,0,0
Politician,0,0,375,4,7,0,0,0,0,1
Writer,6,0,5,66,11,0,2,0,0,0
Book,0,5,3,8,79,0,0,0,0,2
Software,0,0,0,0,0,30,0,0,0,0
Actor,0,0,0,0,0,0,0,0,0,0
Food,0,0,0,0,0,0,0,17,0,0
Animal,0,0,0,0,0,0,0,0,0,0
Disease,0,0,1,0,1,0,0,1,12,23


随着训练数据的增加，可以看到macro、micro的准确率、召回率、F1-score均在逐步提高。根据 confusion matrix ，模型在预测 actor 和 animal 时表现极差，常常把 actor 预测为 film（可能是因为二者本来就有很强的内在联系），把 animal 预测为 disease（猜想是因为动物出现的场景下，常常会带来疾病。）

In [18]:
time2=time.time()
print('%.2fs'%(time2-time1))

2352.41s


## 附录

完整版 kneser_ney 实现。

In [ ]:
class ngrams_model:
    def __init__(self, n, method, vocab):
        self.n = n
        # 预定义 self.dic 中的每个 value 均为 ConditionalFreqDist 类， self.dic 第一层选择 n-gram（应对需要回退或者插值的情况），
        # 第二层选择 context，第三层选择 word
        self.prefix = defaultdict(ConditionalFreqDist)
        # unigram 不需要 context
        self.uni = FreqDist()
        self.vocab = vocab
        if method in ['add_one', 'kneser_ney']:
            self.method = method
        else:
            raise 'Method is restricted in `Add-one` or `Kneser-Ney`'

    @staticmethod
    def pad(sent, k):
        if k==1:
            return sent+['<end>']
        pad1 = ['_'] * (k - 1) + ['<begin>']
        pad2 = ['<end>']
        return pad1 + sent + pad2

    def fit(self, dataset):
        # 得到各个长度的语言模型上的 context 和 word 的对应关系
        if self.method == 'add_one':
            k = self.n
            for text in dataset:
                for sent in text:
                    # 在每句话前后加 padding
                    sent = ngrams_model.pad(sent, k)
                    for i in range(len(sent) - k + 1):
                        gram = tuple(sent[i:i + k])
                        if k == 1:
                            self.uni[gram[0]] += 1
                        else:
                            self.prefix[k][gram[:-1]][gram[-1]] += 1
        elif self.method == 'kneser_ney':
            self.suffix = defaultdict(ConditionalFreqDist)
            self.total_gram=defaultdict(int)
            self.middle=defaultdict(ConditionalFreqDist)
            for k in range(self.n,0,-1):
                for text in dataset:
                    for sent in text:
                        # 在每句话前后加 padding
                        sent = ngrams_model.pad(sent, k)
                        for i in range(len(sent) - k + 1):
                            gram = tuple(sent[i:i + k])
                            if k == 1:
                                self.uni[gram[0]] += 1
                            else:
                                self.prefix[k][gram[:-1]][gram[-1]] += 1
                                self.suffix[k][gram[1:]][gram[0]] += 1
                            if k>=3:
                                # 为后面计算 C_kn 做准备
                                self.middle[k][gram[1:-1]][(gram[0],gram[-1])]=1
                self.total_gram[k] = sum(dic.B() for dic in self.prefix[k].values())

    def add_one(self, context, word):
        n = len(context) + 1
        if n == 1:
            dic = self.uni
        else:
            dic = self.prefix[n][context]
        # 进行 add-1 平滑
        up = dic[word] + 1
        down = dic.N() + len(self.vocab)
        return up / down

    def kneser_ney(self, context, word, d=0.1):
        n = len(context) + 1
        if n == 1:
            # P_continuation()
            up = self.suffix[2][(word,)].B()
            down = self.total_gram[2]
            res = (up + 1) / (down + len(self.vocab))
            return res
        if n == self.n:
            dic = self.prefix[n][context]
            if dic[word]:
                a = max(0, dic[word] - d) / dic.N()
            else:
                a = 0
            if dic.B() == 0:
                lam = 1
            else:
                lam = d * dic.B() / dic.N()
        else:
            # Continuation count(x)= Number of unique single word contexts for x
            context_plus = tuple(list(context) + [word])
            c_kn_up = self.suffix[n + 1][context_plus].B()
            c_kn_down = self.total_gram[n+1]
            if c_kn_up:
                a = max(0, c_kn_up - d) / c_kn_down
            else:
                a = 0
            # 计算 q，使得对 w_i 求和为 1
            q = self.middle[n+1][context[1:]].B()
            if q == 0:
                lam = 1
            else:
                lam = d*q/c_kn_down
        return a + lam * self.kneser_ney(context[1:], word, d)  # 递归

    def perplexity(self, sent):
        # 计算困惑度
        # 如果输入为 string，对其进行 preprocessing 以及分词
        if isinstance(sent, str):
            sent = del_blank(word_tokenize(preprocess(sent)))
        sent_ = ngrams_model.pad(sent, self.n)
        ngrams_ = [tuple(sent_[i:i + self.n]) for i in range(len(sent_) - self.n + 1)]
        # 计算 log probability，防止数据下溢
        log_prob = -sum([log(getattr(self, self.method)(gram[:-1], gram[-1]), 2) for gram in ngrams_]) / len(sent)
        return pow(log_prob, 2)  # 由 log probabilitu 转换回 probability

    def test(self, dataset):
        # 计算测试集各句子平均困惑度
        res = []
        for text in dataset:
            for sent in text:
                res.append(self.perplexity(sent))
        return sum(res) / len(res)

    def generate(self, early_stop=None):
        # 生成句子
        # 设置初始 context
        context = tuple(['_'] * (self.n - 2) + ['<begin>'])[:self.n - 1]
        gram = '<begin>'
        sent = []
        keys = list(self.vocab)
        i = 0
        while gram != '<end>':
            # 获得一个 [0,1] 的概率
            p = np.random.rand()
            # 获得词汇表中所有词的概率 pdf
            pdf = []
            for key in keys:
                pdf.append(getattr(self, self.method)(context, key))
            # 累加得到 cdf
            cdf = np.array(pdf).cumsum()
            # 找到随机概率对应的 word
            gram = keys[cdf.searchsorted(p)]
            sent.append(gram)
            context = tuple(list(context[1:]) + [gram])
            i += 1
            # 是否需要提前停止
            if early_stop is not None and i >= early_stop:
                sent.append('<end>')
                break
        # 去除最后的 <end> 
        sent = sent[:-1]
        return sent

    def __call__(self, *args, **kwargs):
        # 可直接调用模型计算单词概率
        return getattr(self, self.method)(*args, **kwargs)